# Construcción del modelo

In [1]:
import torch
print("¿CUDA disponible?", torch.cuda.is_available())
print("GPU actual:", torch.cuda.get_device_name(0))

¿CUDA disponible? True
GPU actual: NVIDIA GeForce RTX 4070 Ti


In [2]:
import os
import cv2
import torch
import face_recognition
import numpy as np
from pathlib import Path
from tqdm import tqdm
from torchvision import transforms
import random
import time
from tqdm import tqdm  # asegúrate de tener esto importado arriba

Crear dataset personalizado

In [3]:
from torch.utils.data import Dataset
import torch

class TensorVideoDataset(Dataset):
    def __init__(self, tensor_paths, labels):
        self.tensor_paths = tensor_paths
        self.labels = labels

    def __len__(self):
        return len(self.tensor_paths)

    def __getitem__(self, idx):
        start = time.time()
        video_tensor = torch.load(self.tensor_paths[idx])
        label = torch.tensor(self.labels[idx]).float()
        #print(f"⏱ Tiempo carga tensor: {time.time() - start:.2f} s")
        return video_tensor, label


Cargar archivos .pt y generar las etiquetas

In [4]:
from pathlib import Path
from sklearn.model_selection import train_test_split

# Ruta donde guardaste los tensores
tensor_dir = Path("C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesados")

# Cargar todos los tensores
all_tensors = list(tensor_dir.glob("*.pt"))

# Etiquetar: 'original' = 0 (real), todo lo demás = 1 (fake)
tensor_labels = [0 if "original" in str(p.name).lower() else 1 for p in all_tensors]

# Dividir en train, val y test (70/15/15)
X_train, X_temp, y_train, y_temp = train_test_split(all_tensors, tensor_labels, test_size=0.3, stratify=tensor_labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


Crear dataloaders

In [11]:
from torch.utils.data import DataLoader

batch_size = 16

train_ds = TensorVideoDataset(X_train, y_train)
val_ds = TensorVideoDataset(X_val, y_val)
test_ds = TensorVideoDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False,  pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False,  pin_memory=True)


Definir el modelo EfficientNet fine-tuneado

In [9]:
from torchvision.models import efficientnet_b0
import torch.nn as nn

class FramewiseEfficientNet(nn.Module):
    def __init__(self, sequence_length=16, dropout=0.4):
        super(FramewiseEfficientNet, self).__init__()
        self.backbone = efficientnet_b0(weights='IMAGENET1K_V1')
        self.backbone.classifier = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 1)
        )

    def forward(self, x):  # x: [B, T, C, H, W]
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        feats = self.backbone(x)
        feats = feats.view(B, T, -1).mean(1)
        return self.classifier(feats).view(-1)


Función train_model

In [7]:
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score
import torch
import torch.nn as nn

def train_model(model, train_loader, val_loader, test_loader,
                lr=1e-4, epochs=10, device='cuda', save_path='best_model.pt'):

    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_auc = 0.0

    def evaluate(loader):
        model.eval()
        y_true, y_scores = [], []
        with torch.no_grad():
            for x, y in loader:
                x = x.to(device)
                y = y.to(device).float()
                logits = model(x)
                probs = torch.sigmoid(logits)
                y_scores.extend(probs.cpu().numpy())
                y_true.extend(y.cpu().numpy())

        y_scores = np.array(y_scores)
        y_true = np.array(y_true)
        y_pred = (y_scores > 0.5).astype(int)

        auc = roc_auc_score(y_true, y_scores)
        acc = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

        return {
            'auc': auc,
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }

    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0

        for i, (x, y) in enumerate(tqdm(train_loader, desc=f"🔁 Epoch {epoch}/{epochs}")):
            x = x.to(device)
            y = y.to(device).float()

            # Solo imprimir en la primera iteración
            if epoch == 1 and i == 0:
                print(f"x device: {x.device} | y device: {y.device} | model device: {next(model.parameters()).device}")

            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        val_metrics = evaluate(val_loader)
        print(f"📘 Epoch {epoch}/{epochs} | Loss: {epoch_loss:.4f} | Val AUC: {val_metrics['auc']:.4f} | Acc: {val_metrics['accuracy']:.4f}")

        # Guardar el mejor modelo
        if val_metrics['auc'] > best_auc:
            best_auc = val_metrics['auc']
            torch.save(model.state_dict(), save_path)
            print("✅ Nuevo mejor modelo guardado.")

    # Evaluar el mejor modelo en test
    print("\n🔍 Evaluando el mejor modelo en el conjunto de prueba...")
    model.load_state_dict(torch.load(save_path))
    test_metrics = evaluate(test_loader)

    print(f"\n📊 Resultados en Test:")
    for k, v in test_metrics.items():
        print(f"{k.capitalize()}: {v:.4f}")

    return test_metrics


Ejecutar el entrenamiento

In [10]:
model = FramewiseEfficientNet()

# 👇 AQUI colocas la prueba del batch
#print("🔍 Probando un batch...")
#x_test, y_test = next(iter(train_loader))
#print(f"Lote cargado correctamente. Forma: {x_test.shape}")

# Si pasa la prueba, se entrena el modelo
metrics = train_model(
    model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    lr=1e-4,
    epochs=10,
    save_path="modelo_finetuneado.pt"
)

🔁 Epoch 1/10:   0%|          | 0/77 [00:00<?, ?it/s]

x device: cuda:0 | y device: cuda:0 | model device: cuda:0


🔁 Epoch 1/10: 100%|██████████| 77/77 [08:54<00:00,  6.94s/it]


📘 Epoch 1/10 | Loss: 30.1072 | Val AUC: 0.8539 | Acc: 0.8664
✅ Nuevo mejor modelo guardado.


🔁 Epoch 2/10: 100%|██████████| 77/77 [09:01<00:00,  7.03s/it]


...

📘 Epoch 8/10 | Loss: 3.5700 | Val AUC: 0.9103 | Acc: 0.9122


🔁 Epoch 9/10: 100%|██████████| 77/77 [08:56<00:00,  6.97s/it]


📘 Epoch 9/10 | Loss: 1.5499 | Val AUC: 0.9208 | Acc: 0.9046


🔁 Epoch 10/10: 100%|██████████| 77/77 [08:56<00:00,  6.97s/it]


📘 Epoch 10/10 | Loss: 1.9455 | Val AUC: 0.9262 | Acc: 0.9351

🔍 Evaluando el mejor modelo en el conjunto de prueba...

📊 Resultados en Test:
Auc: 0.9207
Accuracy: 0.9163
Precision: 0.9432
Recall: 0.9600
F1: 0.9515


In [14]:
# ============================
# Evaluación en el conjunto de prueba
# ============================

from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_fscore_support
import numpy as np

device = "cuda"

model = FramewiseEfficientNet()
model.load_state_dict(torch.load("modelo_finetuneado.pt"))
model = model.to(device)  # 💥 Muy importante mover el modelo a GPU DESPUÉS de cargar pesos
model.eval()


y_true, y_scores = [], []


with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        y = y.to(device).float()
        logits = model(x)
        probs = torch.sigmoid(logits)
        y_scores.extend(probs.cpu().numpy())
        y_true.extend(y.cpu().numpy())

y_scores = np.array(y_scores)
y_true = np.array(y_true)
y_pred = (y_scores > 0.5).astype(int)

auc = roc_auc_score(y_true, y_scores)
acc = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

print("\n📊 Resultados en el conjunto de prueba:")
print(f"AUC:       {auc:.4f}")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")



📊 Resultados en el conjunto de prueba:
AUC:       0.9511
Accuracy:  0.9237
Precision: 0.9476
Recall:    0.9644
F1 Score:  0.9559
